In [1]:
import RandomForest as RF
import os.path
import os
from datetime import datetime
import bz2
import _pickle as cPickle
import numpy as np
import random
import csv
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings("ignore")

my_os = "mac" #win mac

separator = "\\"

if my_os == "mac":
    separator = "//"

division = "subsets" # subsets random_equal random_same

today = datetime.now()

if division == "subsets":
    folderName =  'Experiments 2 '  + today.strftime("%d-%b-%Y %H_%M_%S")
else:
    folderName =  'Experiments 2 ' + division + ' '  + today.strftime("%d-%b-%Y %H_%M_%S")

if not os.path.exists(folderName):
    os.makedirs(folderName)

dataset = "kdd" # iris kdd  nsl_kdd  unsw_nb15 cic_ids_2017
problem = "AC" # AD AC
TrainingPercentage = 0.7
ValidationPercentage = 0.1
TestingPercentage = 0.2



no_runs = 5

results_2_1 = []
results_2_2 = []
results_2_3 = []
results_2_4 = []

k = 1
while k<=no_runs:
    # Dividing the dataset
    print("RUN " + str(k))

    if dataset == "iris":
        iris = load_iris()
        data, target = iris.data, iris.target

    if dataset == "kdd":
        file_name = "data" + separator + "kdd_subsets.pbz2"
        f = bz2.BZ2File(file_name, "rb")
        data, target, subsets = cPickle.load(f)
        f.close()
        label_for_normal = 3


    if dataset == "nsl_kdd":
        file_name = "data" + separator + "nsl_kdd_subsets.pbz2"
        f = bz2.BZ2File(file_name, "rb")
        data, target, subsets = cPickle.load(f)
        f.close()
        label_for_normal = 4

    if dataset == "unsw_nb15":
        file_name = "data" + separator + "unsw_nb15_subsets.pbz2"
        f = bz2.BZ2File(file_name, "rb")
        data, target, subsets = cPickle.load(f)
        f.close()
        label_for_normal = 4

    if dataset == "cic_ids_2017":
        file_name = "data" + separator +  "cic_ids_2017_subsets.pbz2"
        f = bz2.BZ2File(file_name, "rb")
        data, target, subsets = cPickle.load(f)
        f.close()
        label_for_normal = 0

    if problem == "AD" and dataset != "iris":
        target[target != label_for_normal] = 1
        target[target == label_for_normal] = 0

    num_classes = np.unique(target).shape[0]


    uniqueSubsets = np.unique(subsets)

    total_number_instances = len(data)
    indexes = range(len(data))
    indexes = random.sample(indexes, round(TrainingPercentage * total_number_instances))
    dataTraining = data[indexes,:]
    targetTraining = target[indexes]
    subsetsTraining = subsets[indexes]
    data = np.delete(data, indexes, axis = 0)
    target = np.delete(target, indexes, axis = 0)
    subsets = np.delete(subsets, indexes, axis = 0)
    indexes = range(len(data))
    indexes = random.sample(indexes, round(ValidationPercentage * total_number_instances))
    dataValidation = data[indexes,:]
    targetValidation = target[indexes]
    subsetsValidation = subsets[indexes]
    dataTesting = np.delete(data, indexes, axis = 0)
    targetTesting = np.delete(target, indexes, axis = 0)
    subsetsTesting = np.delete(subsets, indexes, axis = 0)

    algorithms = []

    percentage = 0.7
    if dataset == "kdd" and problem == "AD" :
        num_trees = 65
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "SV"

    if dataset == "kdd" and problem == "AC" :
        num_trees = 53
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "WV"

    if dataset == "nsl_kdd" and problem == "AD" :
        num_trees = 65
        criterion_input = 'gini' #["gini", "entropy"]
        ensemble_method = "SV"

    if dataset == "nsl_kdd" and problem == "AC" :
        num_trees = 31
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "SV"

    if dataset == "unsw_nb15" and problem == "AD" :
        num_trees = 93
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "WV"

    if dataset == "unsw_nb15" and problem == "AC" :
        num_trees = 61
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "SV"

    if dataset == "cic_ids_2017" and problem == "AD" :
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "WV"
        num_trees = 77

    if dataset == "cic_ids_2017" and problem == "AC" :
        criterion_input = 'entropy' #["gini", "entropy"]
        ensemble_method = "WV"
        num_trees = 95

    ok = 1

    for i in range(len(uniqueSubsets)):
        temp_result = ""
        ind_list = np.where(subsetsTraining == uniqueSubsets[i])
        subsetDataTraining = dataTraining[ind_list]
        subsetTargetTraining = targetTraining[ind_list]
        ind_list = np.where(subsetsValidation == uniqueSubsets[i])
        subsetDataValidation = dataValidation[ind_list]
        subsetTargetValidation = targetValidation[ind_list]
        ind_list = np.where(subsetsTesting == uniqueSubsets[i])
        subsetDataTesting = dataTesting[ind_list]
        subsetTargetTesting = targetTesting[ind_list]
        if subsetDataTraining.shape[0] == 0 or subsetDataValidation.shape[0]==0 or subsetDataTesting.shape[0] == 0:
            print("not ok " + uniqueSubsets[i] + ", again")
            ok = 0

    if ok == 0:
        continue

    f = bz2.BZ2File(folderName+separator+dataset+"_"+problem+"_"+str(k)+ "_subsets_for_Experiment3.pbz2", "wb")
    cPickle.dump((dataTraining,targetTraining,subsetsTraining,dataValidation,targetValidation,subsetsValidation,dataTesting,targetTesting,subsetsTesting),f)
    f.close()

    result = ""
    print("**********************************************")
    print()
    print("         " + str(k)  + "   EXPERIMENT 2.1")
    print()
    print("**********************************************")
    one_run_2_1 = []
    if division == "subsets":
        result = result + "2.1\n\n\n"

        for i in range(len(uniqueSubsets)):
            temp_result = ""
            ind_list = np.where(subsetsTraining == uniqueSubsets[i])
            subsetDataTraining = dataTraining[ind_list]
            subsetTargetTraining = targetTraining[ind_list]
            ind_list = np.where(subsetsValidation == uniqueSubsets[i])
            subsetDataValidation = dataValidation[ind_list]
            subsetTargetValidation = targetValidation[ind_list]
            ind_list = np.where(subsetsTesting == uniqueSubsets[i])
            subsetDataTesting = dataTesting[ind_list]
            subsetTargetTesting = targetTesting[ind_list]

            total_number_instances_subset = subsetDataTraining.shape[0] + subsetDataValidation.shape[0] + subsetDataTesting.shape[0]


            temp_result = temp_result + "\n INFORMATION\n"+"\n"
            temp_result = temp_result + "Dataset: "+dataset+"\n"
            temp_result = temp_result + "Subset: "+str(uniqueSubsets[i])+"\n"
            temp_result = temp_result + "Total number of instances: "+str(total_number_instances_subset)+"\n"
            temp_result = temp_result + "Number Training Cases: "+str(subsetDataTraining.shape[0])+"\n"
            temp_result = temp_result + "Number Validation Cases: "+str(subsetDataValidation.shape[0])+"\n"
            temp_result = temp_result + "Number whole Testing Cases: "+str(subsetDataTesting.shape[0])+"\n"+"\n"
            print(temp_result)

            temp_result = temp_result + "numTrees: "+str(num_trees)+"\n"
            algorithms.append(RF.RandomForest(num_trees, criterion_input, ensemble_method, num_classes))
            algorithms[-1].fit(subsetDataTraining, subsetTargetTraining, percentage)
            algorithms[-1].validate(subsetDataValidation, subsetTargetValidation)
            algorithms[-1].predict(subsetDataTesting, subsetTargetTesting)
            temp_result = temp_result + "\nAccuracy: " + str(algorithms[-1].statisticsRF.accuracy)+ "Precision: " + str(algorithms[-1].statisticsRF.precision) + "Recall: " + str(algorithms[-1].statisticsRF.recall) + "F1: " + str(algorithms[-1].statisticsRF.f1)+"\n"
            one_run_2_1.append(algorithms[-1].statisticsRF)
            print("\nAccuracy: " + str(algorithms[-1].statisticsRF.accuracy)+"\n")
            result = result + temp_result
        results_2_1.append(one_run_2_1)
    print("**********************************************")
    print()
    print("         " + str(k)  + "  EXPERIMENT 2.2")
    print()
    print("**********************************************")

    if division == "subsets":
        result = result + "\n\n\n2.2\n\n\n"
        one_run_2_2 = []
        for i in range(len(uniqueSubsets)):
            temp_result = ""

            ind_list = np.where(subsetsTraining == uniqueSubsets[i])
            subsetDataTraining = dataTraining[ind_list]
            subsetTargetTraining = targetTraining[ind_list]
            ind_list = np.where(subsetsValidation == uniqueSubsets[i])
            subsetDataValidation = dataValidation[ind_list]
            subsetTargetValidation = targetValidation[ind_list]

            total_number_instances_subset = subsetDataTraining.shape[0] + subsetDataValidation.shape[0] + subsetDataTesting.shape[0]


            temp_result = temp_result + "\n INFORMATION\n"+"\n"
            temp_result = temp_result + "Dataset: "+dataset+"\n"
            temp_result = temp_result + "Subset: "+str(uniqueSubsets[i])+"\n"
            temp_result = temp_result + "Total number of instances: "+str(total_number_instances_subset)+"\n"
            temp_result = temp_result + "Number Training Cases: "+str(subsetDataTraining.shape[0])+"\n"
            temp_result = temp_result + "Number Validation Cases: "+str(subsetDataValidation.shape[0])+"\n"
            temp_result = temp_result + "Number whole Testing Cases: "+str(dataTesting.shape[0])+"\n"+"\n"


            temp_result = temp_result + "numTrees: "+str(num_trees)+"\n"

            algorithms[i].predict(dataTesting, targetTesting)
            temp_result = temp_result + "\nAccuracy: " + str(algorithms[i].statisticsRF.accuracy)+ "Precision: " + str(algorithms[i].statisticsRF.precision) + "Recall: " + str(algorithms[i].statisticsRF.recall) + "F1: " + str(algorithms[i].statisticsRF.f1)+"\n"
            one_run_2_2.append(algorithms[i].statisticsRF)
            print(temp_result)
            result = result + temp_result
        results_2_2.append(one_run_2_2)

        f = open(folderName+"\\"+"Experiment_2_2_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method, "wb")
        f = bz2.BZ2File(folderName+separator+"Experiment_2_2_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+ "_" + str(k) + ".pbz2", "wb")
        cPickle.dump((algorithms),f)
        f.close()
        print("Results 2.1")
        print(results_2_1)
        print("Results 2.2")
        print(results_2_2)

    print("**********************************************")
    print()
    print("            EXPERIMENT 2.3")
    print()
    print("**********************************************")

    if division == "random_equal":
        one_run_2_3 = []

        total_subsets =  len(uniqueSubsets)
        per_subset_training = len(dataTraining)/total_subsets
        per_subset_validation = len(dataValidation)/total_subsets

        print("Total training: " + str(len(dataTraining)))
        print("Subsets: " + str(total_subsets))
        print("Per subset: " + str(per_subset_training))

        for i in range(total_subsets):
            temp_result = ""

            indexes = range(len(dataTraining))
            indexes = random.sample(indexes, int(per_subset_training))

            subsetDataTraining = dataTraining[indexes,:]
            subsetTargetTraining = targetTraining[indexes]

            dataTraining = np.delete(dataTraining, indexes, axis = 0)
            targetTraining = np.delete(targetTraining, indexes, axis = 0)

            indexes = range(len(dataValidation))
            indexes = random.sample(indexes, int(per_subset_validation))

            subsetDataValidation = dataValidation[indexes,:]
            subsetTargetValidation = targetValidation[indexes]

            dataValidation = np.delete(dataValidation, indexes, axis = 0)
            targetValidation = np.delete(targetValidation, indexes, axis = 0)
            # total_number_instances_subset = subsetDataTraining.shape[0] + subsetDataValidation.shape[0] + subsetDataTesting.shape[0]

            subset_name = str(i+1);

            temp_result = temp_result + "\n INFORMATION\n"+"\n"
            temp_result = temp_result + "Dataset: "+dataset+"\n"
            temp_result = temp_result + "Subset: "+subset_name+"\n"
            # print("Total number of instances: "+str(total_number_instances_subset))
            temp_result = temp_result +"Number Training Cases: "+str(subsetDataTraining.shape[0])+"\n"
            temp_result = temp_result +"Number Validation Cases: "+str(subsetDataValidation.shape[0])+"\n"
            temp_result = temp_result +"Number Whole Testing Cases: "+str(dataTesting.shape[0])+"\n"+"\n"


            temp_result = temp_result +"numTrees: "+str(num_trees)

            algorithms.append(RF.RandomForest(num_trees, criterion_input, ensemble_method, num_classes))

            algorithms[-1].fit(subsetDataTraining, subsetTargetTraining, percentage)
            algorithms[-1].validate(subsetDataValidation, subsetTargetValidation)
            algorithms[-1].predict(dataTesting, targetTesting)

            temp_result = temp_result + "\nAccuracy: " + str(algorithms[i].statisticsRF.accuracy)+ "Precision: " + str(algorithms[i].statisticsRF.precision) + "Recall: " + str(algorithms[i].statisticsRF.recall) + "F1: " + str(algorithms[i].statisticsRF.f1)+"\n"
            one_run_2_3.append(algorithms[i].statisticsRF)
            print(temp_result)
            result = result + temp_result
        results_2_3.append(one_run_2_3)

        print("Results 2.3")
        print(results_2_3)

        f = bz2.BZ2File(folderName+separator+"Experiment_2_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+ "_" + str(k) + ".pbz2", "wb")
        cPickle.dump(algorithms,f)
        f.close()

    if division == "random_same":
        one_run_2_4 = []
        total_subsets =  len(uniqueSubsets)
        for i in range(total_subsets):
            temp_result = ""

            per_subset_training = len(np.array(np.where(subsetsTraining == uniqueSubsets[i])[0]))
            per_subset_validation =len(np.array(np.where(subsetsValidation == uniqueSubsets[i])[0]))

            indexes = range(len(dataTraining))
            indexes = random.sample(indexes,per_subset_training)

            subsetDataTraining = dataTraining[indexes,:]
            subsetTargetTraining = targetTraining[indexes]

            dataTraining = np.delete(dataTraining, indexes, axis = 0)
            targetTraining = np.delete(targetTraining, indexes, axis = 0)

            indexes = range(len(dataValidation))
            indexes = random.sample(indexes, per_subset_validation)

            subsetDataValidation = dataValidation[indexes,:]
            subsetTargetValidation = targetValidation[indexes]

            dataValidation = np.delete(dataValidation, indexes, axis = 0)
            targetValidation = np.delete(targetValidation, indexes, axis = 0)
            # total_number_instances_subset = subsetDataTraining.shape[0] + subsetDataValidation.shape[0] + subsetDataTesting.shape[0]

            subset_name = str(i+1);

            temp_result = temp_result + "\n INFORMATION\n"+"\n"
            temp_result = temp_result + "Dataset: "+dataset+"\n"
            temp_result = temp_result + "Subset: "+subset_name+"\n"
            # print("Total number of instances: "+str(total_number_instances_subset))
            temp_result = temp_result +"Number Training Cases: "+str(subsetDataTraining.shape[0])+"\n"
            temp_result = temp_result +"Number Validation Cases: "+str(subsetDataValidation.shape[0])+"\n"
            temp_result = temp_result +"Number Whole Testing Cases: "+str(dataTesting.shape[0])+"\n"+"\n"


            temp_result = temp_result +"numTrees: "+str(num_trees)

            algorithms.append(RF.RandomForest(num_trees, criterion_input, ensemble_method, num_classes))

            algorithms[-1].fit(subsetDataTraining, subsetTargetTraining, percentage)
            algorithms[-1].validate(subsetDataValidation, subsetTargetValidation)
            algorithms[-1].predict(dataTesting, targetTesting)

            temp_result = temp_result + "\nAccuracy: " + str(algorithms[i].statisticsRF.accuracy)+ "Precision: " + str(algorithms[i].statisticsRF.precision) + "Recall: " + str(algorithms[i].statisticsRF.recall) + "F1: " + str(algorithms[i].statisticsRF.f1)+"\n"
            one_run_2_4.append(algorithms[i].statisticsRF)
            print(temp_result)
            result = result + temp_result
        results_2_4.append(one_run_2_4)

        print("Results 2.4")
        print(results_2_4)

        f = open(folderName+"\\"+"Experiment_2_1_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method, "wb")
        f = bz2.BZ2File(folderName+separator+"Experiment_2_1_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+".pbz2", "wb")
        cPickle.dump((algorithms),f)
        f.close()


    f1 = open(folderName+separator+"output_"+dataset+"_"+problem+".txt", "w")
    f1.write(result)
    f1.close()

    k = k + 1

if division == "subsets":
    avg_2_1 = []
    avg_2_2 = []

    prec_2_1 = []
    prec_2_2 = []

    recall_2_1 = []
    recall_2_2 = []


    f1_2_1 = []
    f1_2_2 = []
    for k in range(len(uniqueSubsets)):
        sum1a = 0
        sum2a = 0
        sum1p = 0
        sum2p = 0
        sum1r = 0
        sum2r = 0
        sum1f = 0
        sum2f = 0
        for z in range(len(results_2_1)):
           sum1a = sum1a + results_2_1[z][k].accuracy
           sum2a = sum2a + results_2_2[z][k].accuracy
           sum1p = sum1p + results_2_1[z][k].precision
           sum2p = sum2p + results_2_2[z][k].precision
           sum1r = sum1r + results_2_1[z][k].recall
           sum2r = sum2r + results_2_2[z][k].recall
           sum1f = sum1f + results_2_1[z][k].f1
           sum2f = sum2f + results_2_2[z][k].f1
        avg_temp1 = sum1a/no_runs
        avg_temp2 = sum2a/no_runs
        prec_temp1 = sum1p/no_runs
        prec_temp2 = sum2p/no_runs
        r_temp1 = sum1r/no_runs
        r_temp2 = sum2r/no_runs
        f1_temp1 = sum1f/no_runs
        f1_temp2 = sum2f/no_runs
        avg_2_1.append(avg_temp1)
        avg_2_2.append(avg_temp2)
        prec_2_1.append(prec_temp1)
        prec_2_2.append(prec_temp2)
        recall_2_1.append(r_temp1)
        recall_2_2.append(r_temp2)
        f1_2_1.append(f1_temp1)
        f1_2_2.append(f1_temp2)


    f = open(folderName+separator+dataset+"_"+problem+"_""results_2_1.csv", 'w')
    writer = csv.writer(f)
    firstRow  = []
    for k in range(len(uniqueSubsets)):
        firstRow.append(str(uniqueSubsets[k]) + "/accuracy")
        firstRow.append(str(uniqueSubsets[k]) + "/precision")
        firstRow.append(str(uniqueSubsets[k]) + "/recall")
        firstRow.append(str(uniqueSubsets[k]) + "/f1")
    writer.writerow(firstRow)

    for k in range(len(results_2_1)):
        oneRow = []
        for j in range(len(uniqueSubsets)):
            oneRow.append(results_2_1[k][j].accuracy)
            oneRow.append(results_2_1[k][j].precision*100)
            oneRow.append(results_2_1[k][j].recall*100)
            oneRow.append(results_2_1[k][j].f1*100)
        writer.writerow(oneRow)
    lastRow = []
    for j in range(len(uniqueSubsets)):
        lastRow.append(avg_2_1[j])
        lastRow.append(prec_2_1[j]*100)
        lastRow.append(recall_2_1[j]*100)
        lastRow.append(f1_2_1[j]*100)
    writer.writerow(lastRow)
    f.close()

    f = open(folderName+separator+dataset+"_"+problem+"_""results_2_2.csv", 'w')
    writer = csv.writer(f)
    writer.writerow(firstRow)
    for k in range(len(results_2_2)):
        oneRow = []
        for j in range(len(uniqueSubsets)):
            oneRow.append(results_2_2[k][j].accuracy)
            oneRow.append(results_2_2[k][j].precision*100)
            oneRow.append(results_2_2[k][j].recall*100)
            oneRow.append(results_2_2[k][j].f1*100)
        writer.writerow(oneRow)
    lastRow = []
    for j in range(len(uniqueSubsets)):
        lastRow.append(avg_2_2[j])
        lastRow.append(prec_2_2[j]*100)
        lastRow.append(recall_2_2[j]*100)
        lastRow.append(f1_2_2[j]*100)
    writer.writerow(lastRow)
    f.close()


if division == "random_equal":
    avg_2_3 = []
    prec_2_3 = []
    recall_2_3 = []
    f1_2_3 = []

    for k in range(len(uniqueSubsets)):
        sum1a = 0
        sum1p = 0
        sum1r = 0
        sum1f = 0
        for z in range(len(results_2_3)):
            sum1a = sum1a + results_2_3[z][k].accuracy
            sum1p = sum1p + results_2_3[z][k].precision
            sum1r = sum1r + results_2_3[z][k].recall
            sum1f = sum1f + results_2_3[z][k].f1
        avg_temp1 = sum1a/no_runs
        prec_temp1 = sum1p/no_runs
        r_temp1 = sum1r/no_runs
        f1_temp1 = sum1f/no_runs
        avg_2_3.append(avg_temp1)
        prec_2_3.append(prec_temp1)
        recall_2_3.append(r_temp1)
        f1_2_3.append(f1_temp1)

    f = open(folderName+separator+dataset+"_"+problem+"_""results_2_3.csv", 'w')
    writer = csv.writer(f)
    firstRow  = []
    for k in range(len(uniqueSubsets)):
        firstRow.append(str(uniqueSubsets[k]) + "/accuracy")
        firstRow.append(str(uniqueSubsets[k]) + "/precision")
        firstRow.append(str(uniqueSubsets[k]) + "/recall")
        firstRow.append(str(uniqueSubsets[k]) + "/f1")
    writer.writerow(firstRow)

    for k in range(len(results_2_3)):
        oneRow = []
        for j in range(len(uniqueSubsets)):
            oneRow.append(results_2_3[k][j].accuracy)
            oneRow.append(results_2_3[k][j].precision*100)
            oneRow.append(results_2_3[k][j].recall*100)
            oneRow.append(results_2_3[k][j].f1*100)
        writer.writerow(oneRow)
    lastRow = []
    for j in range(len(uniqueSubsets)):
        lastRow.append(avg_2_3[j])
        lastRow.append(prec_2_3[j]*100)
        lastRow.append(recall_2_3[j]*100)
        lastRow.append(f1_2_3[j]*100)
    writer.writerow(lastRow)
    f.close()



if division == "random_same":
    avg_2_4 = []
    prec_2_4 = []
    recall_2_4 = []
    f1_2_4 = []

    for k in range(len(uniqueSubsets)):
        sum1a = 0
        sum1p = 0
        sum1r = 0
        sum1f = 0
        for z in range(len(results_2_4)):
            sum1a = sum1a + results_2_4[z][k].accuracy
            sum1p = sum1p + results_2_4[z][k].precision
            sum1r = sum1r + results_2_4[z][k].recall
            sum1f = sum1f + results_2_4[z][k].f1
        avg_temp1 = sum1a/no_runs
        prec_temp1 = sum1p/no_runs
        r_temp1 = sum1r/no_runs
        f1_temp1 = sum1f/no_runs
        avg_2_4.append(avg_temp1)
        prec_2_4.append(prec_temp1)
        recall_2_4.append(r_temp1)
        f1_2_4.append(f1_temp1)

    f = open(folderName+separator+dataset+"_"+problem+"_""results_2_4.csv", 'w')
    writer = csv.writer(f)
    firstRow  = []
    for k in range(len(uniqueSubsets)):
        firstRow.append(str(k+1) + "/accuracy")
        firstRow.append(str(k+1) + "/precision")
        firstRow.append(str(k+1) + "/recall")
        firstRow.append(str(k+1) + "/f1")
    writer.writerow(firstRow)

    for k in range(len(results_2_4)):
        oneRow = []
        for j in range(len(uniqueSubsets)):
            oneRow.append(results_2_4[k][j].accuracy)
            oneRow.append(results_2_4[k][j].precision*100)
            oneRow.append(results_2_4[k][j].recall*100)
            oneRow.append(results_2_4[k][j].f1*100)
        writer.writerow(oneRow)
    lastRow = []
    for j in range(len(uniqueSubsets)):
        lastRow.append(avg_2_4[j])
        lastRow.append(prec_2_4[j]*100)
        lastRow.append(recall_2_4[j]*100)
        lastRow.append(f1_2_4[j]*100)
    writer.writerow(lastRow)
    f.close()

RUN 1
**********************************************

         1   EXPERIMENT 2.1

**********************************************

 INFORMATION

Dataset: kdd
Subset: icmp
Total number of instances: 283235
Number Training Cases: 198478
Number Validation Cases: 28132
Number whole Testing Cases: 56625


Training finished

Accuracy: 99.98763796909492


 INFORMATION

Dataset: kdd
Subset: tcp
Total number of instances: 189786
Number Training Cases: 132679
Number Validation Cases: 19144
Number whole Testing Cases: 37963


Training finished

Accuracy: 99.87092695519321


 INFORMATION

Dataset: kdd
Subset: udp
Total number of instances: 20326
Number Training Cases: 14186
Number Validation Cases: 2059
Number whole Testing Cases: 4081


Training finished

Accuracy: 99.75496201911295

**********************************************

         1  EXPERIMENT 2.2

**********************************************

 INFORMATION

Dataset: kdd
Subset: icmp
Total number of instances: 230691
Number Training Ca

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/tmc01/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/sy/n7rh139x6tl49pn2wm46j4vd7tmz20/T/ipykernel_21375/4192946438.py", line 257, in <module>
    algorithms[i].predict(dataTesting, targetTesting)
  File "/Users/tmc01/DataspellProjects/paper3/for git/RandomForest.py", line 125, in predict
    results[j,self.statisticsTrees[i].prediction[j]] += self.statisticsTreesValidation[i].accuracyClass[self.statisticsTrees[i].prediction[j]] * np.average(self.statisticsTreesValidation[i].accuracyClass)
  File "<__array_function__ internals>", line 200, in average
  File "/Users/tmc01/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py", line 518, in average
    avg = a.mean(axis, **keepdims_kw)
  File "/Users/tmc01/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py", line 181, in _mean
    ret

TypeError: object of type 'NoneType' has no len()